In [ ]:
!pip install stanza nltk anytree
import nltk
nltk.download('punkt_tab')
import re
import nltk  #imports our natural language toolkit
import csv   #helps us work with csv files
from difflib import SequenceMatcher #helps compare sequences

Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ruchi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


**Reasoning**:
The previous command failed because the `sklearn` library is not installed. I need to install it before I can import `TfidfVectorizer`.



In [22]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   -------- ------------------------------- 1.8/8.7 MB 12.6 MB/s eta 0:00:01
   ---------------------------- ----------- 6.3/8.7 MB 18.3 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 18.1 MB/s  0:00:00

   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-learn]
   -------------------- ------------------- 1/2 [scikit-l

**Reasoning**:
Import the stanza library and download the English language model.



In [48]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
try:
    df = pd.read_csv(r'C:\Users\ruchi\Documents\Dataset.csv')
    original_sentences = df['Original'].tolist()
except FileNotFoundError:
    print("Error: Dataset.csv not found. Please make sure the file path is correct.")
    original_sentences = []
if original_sentences:
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(original_sentences)
    feature_names = tfidf_vectorizer.get_feature_names_out()

    print("TF-IDF calculation complete.")
else:
    print("Could not calculate TF-IDF as no sentences were loaded.")

TF-IDF calculation complete.


In [47]:
tfidf_threshold = 0.1  #THIS IS THE TFIDF THRESHOLD
potential_jargon = set()

if 'tfidf_matrix' in locals() and 'feature_names' in locals():
    for i in range(tfidf_matrix.shape[0]):
        sentence_tfidf_scores = tfidf_matrix[i].toarray()[0]
        for j in range(len(feature_names)):
            score = sentence_tfidf_scores[j]
            if score > tfidf_threshold:
                potential_jargon.add(feature_names[j])

    print(f"Identified {len(potential_jargon)} potential jargon terms.")
    print("Potential jargon examples:", list(potential_jargon)[:20])
else:
    print("TF-IDF matrix or feature names not available. Please run the previous cell.")

Identified 2441 potential jargon terms.
Potential jargon examples: ['bigamy', 'epilepsy', 'false', 'third', 'trace', 'relational', 'admission', 'ordered', 'files', 'adjudication', 'redress', 'redirect', 'storage', 'address', 'left', 'correctly', 'commuted', 'algorithm', 'artificial', 'systems']


In [46]:
import stanza
import nltk
from nltk.corpus import wordnet as wn #DOWNLOADIGN THE WORDNET DICTIONARY

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')
# Download the English language model for Stanza
try:
    stanza.download('en')
except Exception as e:
    print(f"Error downloading Stanza model: {e}")

try:
    nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')  #INITIALIZES THE PARSING PIPELINE
    print("Stanza pipeline initialized.")
except Exception as e:
    print(f"Error initializing Stanza pipeline: {e}")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ruchi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2025-08-25 18:37:23 INFO: Downloaded file to C:\Users\ruchi\stanza_resources\resources.json
2025-08-25 18:37:23 INFO: Downloading default packages for language: en (English) ...
2025-08-25 18:37:24 INFO: File exists: C:\Users\ruchi\stanza_resources\en\default.zip
2025-08-25 18:37:28 INFO: Finished downloading models and saved to C:\Users\ruchi\stanza_resources
2025-08-25 18:37:28 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-08-25 18:37:28 INFO: Downloaded file to C:\Users\ruchi\stanza_resources\resources.json
2025-08-25 18:37:29 INFO: Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

2025-08-25 18:37:29 INFO: Using device: cpu
2025-08-25 18:37:29 INFO: Loading: tokenize
2025-08-25 18:37:29 INFO: Loading: mwt
2025-08-25 18:37:29 INFO: Loading: pos
2025-08-25 18:37:30 INFO: Loading: lemma
2025-08-25 18:37:31 INFO: Loading: depparse
2025-08-25 18:37:31 INFO: Done loading processors!


Stanza pipeline initialized.


In [45]:
def simplify_sentence_with_wordnet(sentence, potential_jargon_list, nlp_pipeline):
    doc = nlp_pipeline(sentence)
    simplified_sentence = []

    for sent in doc.sentences:
        for word in sent.words:
            original_word = word.text
            lemma_word = word.lemma
            simplified_word = original_word
            if original_word.lower() in potential_jargon_list or lemma_word.lower() in potential_jargon_list: #CHECKS IF THE WORD IS IN THE JARGON LIST
                synonyms = [] #CREATES A LIST OF SYNONYMS
                for syn in wn.synsets(lemma_word):
                    for lemma in syn.lemmas():
                        synonyms.append(lemma.name().replace('_', ' '))

                synonyms.sort(key=len) #THIS LINE PRIORITIZES SHORTER SYNONYMS INSTEAD
                if synonyms:
                    simplified_word = synonyms[0]


            simplified_sentence.append(simplified_word)





    return " ".join(simplified_sentence)



In [44]:
test_sentence = "The plaintiff filed a motion to dismiss the case."
simplified_example = simplify_sentence_with_wordnet(test_sentence, potential_jargon, nlp)
print(f"Original: {test_sentence}")
print(f"Simplified: {simplified_example}")

test_sentence_2 = "The court decided to abate the lawsuit."
simplified_example_2 = simplify_sentence_with_wordnet(test_sentence_2, potential_jargon, nlp)
print(f"Original: {test_sentence_2}")
print(f"Simplified: {simplified_example_2}")

test_sentence_3 = "The evidence showed the defendant did abet the crime by providing a getaway car."
simplified_example_3 = simplify_sentence_with_wordnet(test_sentence_3, potential_jargon, nlp)
print(f"Original: {test_sentence_3}")
print(f"Simplified: {simplified_example_3}")

Original: The plaintiff filed a motion to dismiss the case.
Simplified: The plaintiff file a move to can the case .
Original: The court decided to abate the lawsuit.
Simplified: The woo decide to slake the suit .
Original: The evidence showed the defendant did abet the crime by providing a getaway car.
Simplified: The show show the suspect do abet the crime by ply a lam car .


In [49]:
from nltk.translate.bleu_score import corpus_bleu

# Assuming 'df' is your DataFrame with 'Original' and 'Simplified' columns
# and 'potential_jargon' and 'nlp' are available from previous cells.

if 'df' in locals() and 'potential_jargon' in locals() and 'nlp' in locals():
    original_sentences = df['Original'].tolist()
    reference_sentences = df['Simplified'].tolist()

    # Generate simplified sentences using the developed function
    candidate_sentences = [simplify_sentence_with_wordnet(sent, potential_jargon, nlp) for sent in original_sentences]

    # Prepare the reference sentences for corpus_bleu
    # corpus_bleu expects a list of lists of reference sentences for each candidate
    # Since we have one reference per original sentence, we wrap each reference in a list
    wrapped_references = [[ref] for ref in reference_sentences]

    # Calculate the corpus BLEU score
    # Note: BLEU score is typically used for machine translation and might not be
    # the most appropriate metric for simplification, but we can calculate it.
    try:
        bleu_score = corpus_bleu(wrapped_references, candidate_sentences)
        print(f"Corpus BLEU score: {bleu_score}")
    except ZeroDivisionError:
        print("Could not calculate BLEU score. This might happen if there are issues with sentence tokenization or very short sentences.")

else:
    print("Required variables (df, potential_jargon, nlp) not found. Please run the previous cells.")

Corpus BLEU score: 0.3057807368286832


In [56]:
# Assuming 'df' is your DataFrame with 'Original' column
# and 'potential_jargon' and 'nlp' are available from previous cells.

if 'df' in locals() and 'potential_jargon' in locals() and 'nlp' in locals():
    original_sentences = df['Original'].tolist()

    # Generate simplified sentences using the developed function
    simplified_sentences_model = [simplify_sentence_with_wordnet(sent, potential_jargon, nlp) for sent in original_sentences]

    # Create a new DataFrame with original and model-simplified sentences
    submission_df = pd.DataFrame({
        'Original': original_sentences,
        'Model Simplified': simplified_sentences_model
    })

    # Save the DataFrame to a new CSV file in a writable directory
    submission_df.to_csv('/content/submission.csv', index=False)

    print("Successfully created '/content/submission.csv' with original and model-simplified sentences.")
else:
    print("Required variables (df, potential_jargon, nlp) not found. Please run the previous cells.")

Successfully created '/content/submission.csv' with original and model-simplified sentences.
